In [1]:
import pandas as pd

In [159]:
df_16_9 = pd.read_csv('./지하철 노선별 시간대별 승하차 인원수/서울교통공사_9호선2-3단계역별일별시간대별승하차인원_20161231.csv', encoding='cp949')

In [160]:
df_16_9.head()

,날짜,호선,역번호,역명,구분,00~01,01~02,02~03,03~04,04~05,...,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~24
0,2016-01-01,9호선,4126,언주,순승차,0,0,0,0,4,...,121,168,156,147,162,100,110,130,92,31
1,2016-01-01,9호선,4126,언주,순하차,2,0,0,0,0,...,113,110,135,135,117,96,127,154,131,99
2,2016-01-02,9호선,4126,언주,순승차,0,0,0,0,0,...,245,239,238,291,227,166,134,139,135,61
3,2016-01-02,9호선,4126,언주,순하차,8,0,0,0,0,...,194,181,174,185,186,135,182,144,131,108
4,2016-01-03,9호선,4126,언주,순승차,0,0,0,0,0,...,219,221,183,209,178,137,124,103,115,56


In [136]:
df_16_9.drop('역번호', axis=1, inplace=True) # 필요없는 column 삭제

In [161]:
df_16_9['호선'] = df_16_9['호선'].apply(lambda x : x[0]) # '9호선' -> '9'로 변경

In [162]:
# 합쳐질 데이터와 column명 통일화
df_16_9_to5 = df_16_9[['00~01','01~02','02~03','03~04','04~05','23~24']] 
df_16_9['23~'] = df_16_9_to5.sum(axis='columns')

In [163]:
# 필요없는 column 삭제
df_16_9.drop(columns=['역번호', '00~01','01~02','02~03','03~04','04~05','23~24'], inplace=True)

In [164]:
df_16_9['구분'] = df_16_9['구분'].apply(lambda x : x[-2:]) # '구분' column을 '승차', '하차'로 변경

In [165]:
df_16_9.head()

,날짜,호선,역명,구분,05~06,06~07,07~08,08~09,09~10,10~11,...,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~
0,2016-01-01,9,언주,승차,50,82,66,75,97,128,...,121,168,156,147,162,100,110,130,92,35
1,2016-01-01,9,언주,하차,16,53,47,66,78,86,...,113,110,135,135,117,96,127,154,131,101
2,2016-01-02,9,언주,승차,31,31,61,117,155,204,...,245,239,238,291,227,166,134,139,135,61
3,2016-01-02,9,언주,하차,10,101,121,189,193,215,...,194,181,174,185,186,135,182,144,131,116
4,2016-01-03,9,언주,승차,28,45,43,103,92,153,...,219,221,183,209,178,137,124,103,115,56


In [148]:
# 순승차와 환승승차, 순하차와 환승하차 병합
for i in range(0, len(df_16_9)-1, 2):
    if df_16_9.loc[i]['구분'] == df_16_9.loc[i+1]['구분']:
        df_16_9.loc[i] = df_16_9.loc[i] + df_16_9.loc[i+1]
        
        day = df_16_9.loc[i, '날짜'][:10]
        df_16_9.loc[i, '날짜'] = day
        
        df_16_9.loc[i, '호선'] = '9'
        
        station_len = int(len(df_16_9.loc[i, '역명'])/2)
        station = df_16_9.loc[i, '역명'][:station_len]
        df_16_9.loc[i, '역명'] = station
        
        dist = df_16_9.loc[i, '구분'][:2]
        df_16_9.loc[i, '구분'] = dist
        
        df_16_9.drop(index = i+1, inplace=True)

In [157]:
# 인덱스 정리
df_16_9 = df_16_9.set_index('날짜').reset_index()

In [158]:
df_16_9

,날짜,호선,역명,구분,05~06,06~07,07~08,08~09,09~10,10~11,...,14~15,15~16,16~17,17~18,18~19,19~20,20~21,21~22,22~23,23~
0,2016-01-01,9,언주,승차,50,82,66,75,97,128,...,121,168,156,147,162,100,110,130,92,35
1,2016-01-01,9,언주,하차,16,53,47,66,78,86,...,113,110,135,135,117,96,127,154,131,101
2,2016-01-02,9,언주,승차,31,31,61,117,155,204,...,245,239,238,291,227,166,134,139,135,61
3,2016-01-02,9,언주,하차,10,101,121,189,193,215,...,194,181,174,185,186,135,182,144,131,116
4,2016-01-03,9,언주,승차,28,45,43,103,92,153,...,219,221,183,209,178,137,124,103,115,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,2016-12-29,9,종합운동장,하차,146,686,1577,3023,2527,1896,...,2237,2607,2767,3504,5378,4400,2309,2869,2981,2677
3656,2016-12-30,9,종합운동장,승차,252,1091,3376,4411,2758,1800,...,2301,2769,3381,4076,4937,3190,2434,2865,2535,1527
3657,2016-12-30,9,종합운동장,하차,161,624,1213,3019,2040,1787,...,2507,3027,3471,4488,5639,4028,2165,2342,2353,2322
3658,2016-12-31,9,종합운동장,승차,210,541,710,1156,1268,1273,...,1789,2068,2399,2448,2196,1846,1839,1777,1793,1913
